In [30]:
import pandas as pd
import numpy as np
import catboost 
from catboost import CatBoostClassifier
import os

In [82]:
def list_folders(model : CatBoostClassifier, dir_path):
    items = os.listdir(dir_path)
    results = []

    for item in items:
        df = pd.read_csv(os.path.join(dir_path,item))
        df['pred'] = model.predict(df)
        drop_intervals = []
        recovery_intervals = []

        start_drop = None
        for i in range(len(df)):
            if df.loc[i, 'pred'] == 1:
                if start_drop is None:
                    start_drop = df.loc[i, 'time']
            else:
                if start_drop is not None:
                    end_drop = df.loc[i-1, 'time']
                    drop_intervals.append([start_drop, end_drop])
                    start_drop = None
        if start_drop is not None:
            end_drop = df.loc[len(df)-1, 'time']
            drop_intervals.append([start_drop, end_drop])

        start_rec = None
        for i in range(len(df)):
            if df.loc[i, 'pred'] == 2:
                if start_rec is None:
                    start_rec = df.loc[i, 'time']
            else:
                if start_rec is not None:
                    end_rec = df.loc[i-1, 'time']
                    recovery_intervals.append([start_rec, end_rec])
                    start_rec = None
        if start_rec is not None:
            end_rec = df.loc[len(df)-1, 'time']
            recovery_intervals.append([start_rec, end_rec])

        results.append({
            'file': item,
            'recovery': recovery_intervals,
            'drop': drop_intervals
        })
        df.to_csv('proba.csv',index=False)
        final_df = pd.DataFrame(results)
    final_df.to_csv('submission.csv',index=False)
    return final_df

In [83]:
dir_path = '/Users/savinovsvatoslav/Code/skvazhina_hack/SiamHack/marked_data'
list_folders(model,dir_path)

,file,recovery,drop
0,1dfaf03c-e297-4d92-a0bf-40b1a829391f.csv,[],"[[0.0, 74.283333]]"
1,1cd4a74a-d7a9-4660-8b55-033a1d0a5892.csv,[],"[[0.0, 360.0], [429.121111, 468.806111], [469...."
2,0d038dc5-922c-47bb-a852-47225352182e.csv,[],"[[0.0, 1.637222], [3.522778, 74.125833]]"
3,0d37c186-2852-42f9-8336-2c73b25402ff.csv,[],"[[0.0, 22.095556], [28.558056, 238.096111], [2..."
4,01aaea9a-ad84-4e6d-9945-48c8b12437a7.csv,[],"[[2547.180556, 2963.9], [4870.4475, 4931.510833]]"
...,...,...,...
92,0cce17b6-74da-4db1-bba3-609f51c73840.csv,[],"[[0.0, 0.386667], [0.979167, 2.006389], [2.562..."
93,1ac6ec34-2f18-4cf9-b972-aff80d8f7c2b.csv,[],"[[0.0, 11.067778], [16.794722, 17.474444], [29..."
94,1c565fd4-b9ec-4c59-a2c0-1a5b8c1361d0.csv,[],"[[7.145278, 12.650556], [33.742222, 33.742222]..."
95,0cd8b373-f6a5-4769-95aa-cbb8459b9395.csv,[],"[[0.0, 16.646944], [17.873056, 74.204167], [29..."


In [64]:
model = CatBoostClassifier()
data = pd.read_csv('/Users/savinovsvatoslav/Code/skvazhina_hack/train_data.csv')
data = data[:100000]
data['class'] = (data['class'] != 0).astype(int)
y = data['class']
x = data.drop('class',axis=1)
model.fit(x,y)

Learning rate set to 0.07361
0:	learn: 0.5395842	total: 7.17ms	remaining: 7.16s
1:	learn: 0.4244992	total: 13.2ms	remaining: 6.57s
2:	learn: 0.3238822	total: 19.7ms	remaining: 6.54s
3:	learn: 0.2415387	total: 25.6ms	remaining: 6.38s
4:	learn: 0.1844507	total: 31.7ms	remaining: 6.31s
5:	learn: 0.1450020	total: 37.4ms	remaining: 6.19s
6:	learn: 0.1186557	total: 42.9ms	remaining: 6.09s
7:	learn: 0.0985271	total: 48.4ms	remaining: 6s
8:	learn: 0.0850353	total: 54.3ms	remaining: 5.98s
9:	learn: 0.0743776	total: 60.4ms	remaining: 5.98s
10:	learn: 0.0663666	total: 66.1ms	remaining: 5.94s
11:	learn: 0.0583003	total: 71.6ms	remaining: 5.89s
12:	learn: 0.0524807	total: 77.5ms	remaining: 5.88s
13:	learn: 0.0473224	total: 83.2ms	remaining: 5.86s
14:	learn: 0.0443424	total: 88.7ms	remaining: 5.82s
15:	learn: 0.0416691	total: 94.4ms	remaining: 5.8s
16:	learn: 0.0391226	total: 99.7ms	remaining: 5.76s
17:	learn: 0.0369520	total: 105ms	remaining: 5.74s
18:	learn: 0.0360788	total: 110ms	remaining: 5.69s